# 第6讲：非线性优化



## 6.1 状态估计问题

### 6.1.1 最大后验与最大似然

SLAM模型由运动方程和观测方程组成：

$$\begin{cases}
\mathbf{x}_{k} = f(\mathbf{x}_{k - 1}, \mathbf{u}_{k}) + \mathbf{w}_{k} \\
\mathbf{z}_{k, j} = h(\mathbf{y}_{j}, \mathbf{x}_{k}) + \mathbf{v}_{k, j} \\
\end{cases} \tag {6-1}$$

其中，$\mathbf{x}_{k}$为相机位姿，可以通过变换矩阵$\mathbf{T}_{k}$或李代数$\exp(\mathbf{\xi}_{k}^{\wedge})$计算。观测方程使用针孔相机模型描述。

* 观测方程

假设机器人在$\mathbf{x}_{k}$处对路标$\mathbf{y}_{j}$进行观测，其在图像中的像素坐标为$\mathbf{z}_{k, j}$，则观测方程表示为：

$$s \mathbf{z}_{k, j} = \mathbf{K} \exp(\mathbf{\xi}_{k}^{\wedge}) \mathbf{y}_{j} \tag {6-2}$$

其中，$\mathbf{K}$为相机内参，$s$表示$\mathbf{y}_{j}$到相机的距离（相机坐标系下$Z$值），$\mathbf{z}_{k, j}$和$\mathbf{y}_{j}$用齐次坐标表示。考虑加噪数据，假设噪声项$\mathbf{w}_{k}$、$\mathbf{v}_{k, j}$满足零均值高斯分布：

$$\mathbf{w}_{k} \sim N(0, \mathbf{R}_{k}), \mathbf{v}_{k, j} \sim N(0, \mathbf{Q}_{k, j}) \tag {6-3}$$

在噪声影响下，通过观测数据$\mathbf{z}$和$\mathbf{u}$推断位姿$\mathbf{x}$和地图$\mathbf{y}$（以及概率分布）是状态估计问题。由于SLAM过程中，这些数据是时间序列，过去的研究者们主要使用滤波器（扩展卡尔曼滤波器，EKF）对其求解。卡尔曼滤波器只关心当前时刻的状态估计$\mathbf{x}_{k}$，对历史状态则不多考虑；相对的，近年来普遍采用的非线性优化方法，使用所有时刻数据进行状态估计，通常认为优于传统滤波器，成为当前视觉SLAM主流方法。

从概率学角度出发，非线性优化中，将全部待估计变量视为一个“状态变量”：

$$\mathbf{x} = \{ \mathbf{x}_{1}, \dots, \mathbf{x}_{N}, \mathbf{y}_{1}, \dots, \mathbf{y}_{M} \}$$

对机器人状态的估计，即是在已知输入数据$\mathbf{y}$和观测数据$\mathbf{z}$的条件下，计算状态$\mathbf{x}$的条件概率分布：

$$P(\mathbf{x} | \mathbf{z}, \mathbf{u}) \tag {6-4}$$

当机器人未装备测量运动的传感器时，只需考虑观测方程，即估计条件概率$P(\mathbf{x} | \mathbf{z}, \mathbf{u})$的
分布。如果忽略图像的时序关系，将它们视为彼此无关的图片，则该问题称为SfM（structure from motion）问题，即从多张图像中重建三维空间结构。根据贝叶斯法则，

$$P(\mathbf{x} | \mathbf{z})
= \frac{P(\mathbf{z} | \mathbf{x}) P(\mathbf{x})}{P(\mathbf{z})}
\propto P(\mathbf{z} | \mathbf{x}) P(\mathbf{x}) \tag {6-5}$$

其中，称$P(\mathbf{x} | \mathbf{z})$为**后验概率**、$P(\mathbf{z} | \mathbf{x})$为**似然**，$P(\mathbf{x})$为**先验**。

后验分布通常难以直接求解，一般采用点估计方式，求解状态的最优估计（在该状态下后验概率最大，即最大后验估计（maximize a posterior，MAP））：

$$\mathbf{x}_{\text{MAP}^{\ast}} = \argmax P(\mathbf{x} | \mathbf{z}) = \argmax P(\mathbf{z} | \mathbf{x}) P(\mathbf{x}) \tag {6-6}$$

*最大后验概率相当于最大化似然和先验的乘积*。当缺少机器人位姿先验信息时，可以求解状态$\mathbf{x}$的最大似然估计（maximize likelihood estimation，MLE）：

$$\mathbf{x}_{\text{MLE}^{\ast}} = \argmax P(\mathbf{z} | \mathbf{x}) \tag {6-7}$$

似然是指“在当前位姿下，可能产生怎样的观测数据”。由于只有观测数据已知，*最大似然估计表示：在何种状态下，最可能产生当前的观测数据*。

### 6.1.2 最小二乘的引出

给定一次观测，在高斯分布假设（$\mathbf{v}_{k, j} \sim N(0, \mathbf{Q}_{k, j})$）下，考虑观测模型：

$$\mathbf{z}_{k, j} = h(\mathbf{y}_{j}, \mathbf{x}_{k}) + \mathbf{v}_{k, j}$$

观测数据的条件概率为：

$$P(\mathbf{z}_{k, j} | \mathbf{x}, \mathbf{y}_{j}) = N(h(\mathbf{y}_{j}, \mathbf{x}_{k}), \mathbf{Q}_{k, j})) \tag {6-8}$$

高斯分布的最大似然可以通过*最小化负对数*方式求解。给定任意高斯分布$\mathbf{x} \sim N(\mathbf{\mu}, \mathbf{\Sigma})$，其概率密度函数为：

$$P(\mathbf{x}) = \frac{1}{\sqrt{(2 \pi)^{N} \det(\Sigma)}} \exp \left(
    - \frac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\text{T}} \mathbf{\Sigma}^{-1} (\mathbf{x} - \mathbf{\mu})
\right) \tag {6-8}$$

其负对数为：

$$- \ln P(\mathbf{x})
= \frac{1}{2} \ln \left( (2 \pi)^{N} \det(\Sigma) \right) + 
\frac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\text{T}} \mathbf{\Sigma}^{-1} (\mathbf{x} - \mathbf{\mu})
\tag {6-9}$$

方程（6-8）SLAM观测模型的状态最大似然估计为

$$\mathbf{x}^{\ast} = \argmin_{\mathbf{x}} (\mathbf{z}_{k, j} - h(\mathbf{y}_{j}, \mathbf{x}_{k}))^{\text{T}} \mathbf{Q}_{k, j}^{-1} (\mathbf{z}_{k, j} - h(\mathbf{y}_{j}, \mathbf{x}_{k})) \tag {6-10}$$

方程（6-10）等价于在$\mathbf{\Sigma}$范数意义下最小化噪声项（误差）的平方。定义运动、观测数据与估计值之间的误差为：

$$\begin{cases}
\mathbf{e}_{v, k} = \mathbf{x}_{k} - f(\mathbf{x}_{k - 1}, \mathbf{u}_{k}) \\
\mathbf{e}_{y, j, k} = \mathbf{z}_{k, j} - h(\mathbf{y}_{j}, \mathbf{x}_{k}) \\
\end{cases} \tag {6-11}$$

该误差的平方和为：

$$\mathcal{L}(\mathbf{x})
= \sum_{k} \mathbf{e}_{v, k}^{\text{T}} \mathbf{R}_{k}^{-1} \mathbf{e}_{v, k} +
\sum_{k} \sum_{j} \mathbf{e}_{y, j, k}^{\text{T}} \mathbf{Q}_{k, j}^{-1} \mathbf{e}_{y, j, k}
\tag {6-12}$$


方程（6-12）为总体意义下SLAM的最小二乘问题（least square problem），其最优解等价于状态的最大似然估计。SLAM最小二乘问题的特定结构：

1. 目标函数由多个误差（加权）平方和组成。即使总体状态变量维数很高，但各误差项形式简单，仅与少数状态变量有关。例如，运动误差仅与$\mathbf{x}_{k - 1}$、$\mathbf{x}_{k}$相关，观测误差仅与$\mathbf{x}_{k}$、$\mathbf{y}_{j}$相关。每个误差项是一个小规模的约束，能够进行线性近似，最后将该误差项的雅可比矩阵块拼接到整体雅可比矩阵中。每个误差项对应的优化变量称为**参数块（parameter block）**。

2. 整体误差由大量小规模型误差项组成，其增量方程求解具有稀疏性，因此在大规模时亦可求解。

3. 使用旋转矩阵（变换矩阵）描述位姿，会引入旋转矩阵自身的约束。使用李代数表示，可将该问题转化为**无约束**最小二乘问题。

4. 平方形式（二范数）度量误差相当于欧氏空间距离的平方。也可使用其它范数构建优化问题。


## 6.2 非线性最小二乘

最小二乘问题：

$$\min_{\mathbf{x}} \mathcal{L}(\mathbf{x}) = \min_{\mathbf{x}} \frac{1}{2} \| f(\mathbf{x}) \|_{2}^{2} \tag {6-13}$$

其中，自变量$\mathbf{x} \in \R^{n}$，$f \in \R^{m}$为任意$m$维非线性函数。令目标函数$\mathcal{L}(\mathbf{x})$（标量）关于$\mathbf{x}$的导数为零，$\mathbf{x}$的最优值满足：

$$\frac{\partial}{\partial \mathbf{x}} \frac{1}{2} \| f(\mathbf{x}) \|_{2}^{2} = 0 \tag {6-14}$$

求解方程（6-14）可得目标函数的极值（极大、极小值或鞍点）。当最小二乘问题不能直接求解时，可以考虑**迭代**方式，给定初始值，迭代更新优化变量，使目标函数下降：

> 1. 给定初始值$\mathbf{x}_{0}$
>
> 2. 第$k$次迭代，计算增量$\Delta \mathbf{x}_{k}$，使得$\| f(\mathbf{x}_{k} + \Delta \mathbf{x}_{k}) \|_{2}^{2}$达到极小值
>
> 3. 若$\Delta \mathbf{x}_{k}$足够小，则停止迭代
>
> 4. 否则，令$\mathbf{x}_{k + 1} = \mathbf{x}_{k} + \Delta \mathbf{x}_{k}$，重复步骤2

迭代方法的核心在于增量$\Delta \mathbf{x}_{k}$的计算。

### 6.2.1 一阶和二阶梯度法

将目标函数$\mathcal{L}(\mathbf{x})$在$\mathbf{x}$邻域进行泰勒展开：

$$\mathcal{L}(\mathbf{x} + \Delta \mathbf{x})
\propto \| f(\mathbf{x} + \Delta \mathbf{x}) \|_{2}^{2}
\approx \| f(\mathbf{x}) \|_{2}^{2} +
\mathbf{J}(\mathbf{x}) \Delta \mathbf{x} +
\frac{1}{2} \Delta \mathbf{x}^{\text{T}} \mathbf{H}(\mathbf{x}) \Delta \mathbf{x} \tag {6-15}$$

其中，$\mathbf{J}(\mathbf{x})$为$\| f(\mathbf{x}) \|_{2}^{2}$关于$\mathbf{x}$的一阶导数（雅可比矩阵（Jacobi matrix）），$\mathbf{H}(\mathbf{x})$为二阶导数（海塞矩阵（Hessian matrix））。

**增量方程**：给定$\mathbf{x}$，$\mathcal{L}(\mathbf{x} + \Delta \mathbf{x})$关于$\Delta \mathbf{x}$的最小值

* **最速下降法**：仅保留一阶梯度，则增量方程为：

$$\Delta \mathbf{x}^{\ast} = \argmin_{\Delta \mathbf{x}} \| f(\mathbf{x}) \|_{2}^{2} +
\mathbf{J}(\mathbf{x}) \Delta \mathbf{x}$$

令方程右端关于$\Delta \mathbf{x}$的导数为零，可得增量的最优方向为：

$$\frac{\Delta \mathbf{x}^{\ast}}{\| \Delta \mathbf{x}^{\ast} \|} = - \frac{\mathbf{J}^{\text{T}}(\mathbf{x})}{\| \mathbf{J}(\mathbf{x}) \|} \tag {6-16}$$

即以$\lambda$为步长沿梯度反方向计算增量。

* **牛顿法**：保留二阶梯度，则增量方程为：

$$\Delta \mathbf{x}^{\ast} = \argmin_{\Delta \mathbf{x}}
\| f(\mathbf{x}) \|_{2}^{2} +
\mathbf{J}(\mathbf{x}) \Delta \mathbf{x} +
\frac{1}{2} \Delta \mathbf{x}^{\text{T}} \mathbf{H}(\mathbf{x}) \Delta \mathbf{x} \tag {6-17}$$

令方程右端关于$\Delta \mathbf{x}$的导数为零，可得增量的解：

$$\mathbf{H}(\mathbf{x}) \Delta \mathbf{x} = - \mathbf{J}^{\text{T}}(\mathbf{x}) \tag {6-18}$$

一阶（二阶）梯度法需要将函数在迭代点附近进行泰勒展开，并求关于增量的最小值。经泰勒展开后，函数变为多项式，因此计算增量时只需求解线性方程，避免直接对非线性函数求导。然而，最速下降法过于贪心，容易出现锯齿路径，因此迭代次数增加；而牛顿法需要计算目标函数的$\mathbf{H}$矩阵，当问题规模较大时，$\mathbf{H}$矩阵计算困难。

### 6.2.2 高斯牛顿法（Gauss-Newton）

高斯牛顿法（Gauss-Newton）是最简单的最优化算法之一，其思想为将$f(\mathbf{x})$（不是目标函数$\mathcal{L}(\mathbf{x})$）进行一阶泰勒展开：

$$f(\mathbf{x} + \Delta \mathbf{x}) \approx f(\mathbf{x}) + \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} \tag {6-19}$$

其中，$\mathbf{J}(\mathbf{x})$为$f(\mathbf{x})$关于$\mathbf{x}$的导数，是一个$m \times n$雅可比矩阵。

根据最小二乘迭代框架，求解目标为计算下降矢量$\Delta \mathbf{x}$，使得$\| f(\mathbf{x} + \Delta \mathbf{x}) \|_{2}^{2}$最小。考虑$f(\mathbf{x})$的一阶泰勒展开，则非线性最小二乘问题转化为线性的最小二乘问题：

$$\begin{aligned}
\Delta \mathbf{x}^{\ast}
& = \argmin_{\Delta \mathbf{x}} \frac{1}{2} \| f(\mathbf{x} + \Delta \mathbf{x}) \|_{2}^{2} \\
& \approx \argmin_{\Delta \mathbf{x}} \frac{1}{2} \| f(\mathbf{x}) + \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} \|_{2}^{2}
\end{aligned} \tag {6-20}$$

计算目标函数$\mathcal{L}(\mathbf{x})$关于对$\Delta \mathbf{x}$的导数，并令其为零，可得线性方程组。

$$\begin{aligned}
\mathcal{L}(\mathbf{x} + \Delta \mathbf{x})
& \approx \frac{1}{2} \| f(\mathbf{x}) + \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} \|_{2}^{2} \\
& = \frac{1}{2} \left( f(\mathbf{x}) + \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} \right)^{\text{T}} \left( f(\mathbf{x}) + \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} \right) \\
& = \frac{1}{2} \left(
    \| f(\mathbf{x}) \|_{2}^{2} +
    2 f^{\text{T}}(\mathbf{x}) \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} +
    \Delta \mathbf{x}^{\text{T}} \mathbf{J}^{\text{T}}(\mathbf{x}) \mathbf{J}(\mathbf{x}) \Delta \mathbf{x}
\right)
\end {aligned}$$

计算$\mathcal{L}(\mathbf{x} + \Delta \mathbf{x})$关于$\Delta \mathbf{x}$的导数并令其为零：

$$\begin{aligned}
\frac{\partial \mathcal{L}(\mathbf{x} + \Delta \mathbf{x})}{\partial \Delta \mathbf{x}}
= 2 \mathbf{J}^{\text{T}}(\mathbf{x}) f(\mathbf{x}) + 2 \mathbf{J}^{\text{T}}(\mathbf{x}) \mathbf{J}(\mathbf{x}) \Delta \mathbf{x}
= \mathbf{0}
\end{aligned}$$

则

$$\mathbf{J}^{\text{T}}(\mathbf{x}) \mathbf{J}(\mathbf{x}) \Delta \mathbf{x} = - \mathbf{J}^{\text{T}}(\mathbf{x}) f(\mathbf{x}) \tag {6-21}$$

方程（6-21）为关于变量$\Delta \mathbf{x}$的*线性方程组*，称为**增量方程**（**高斯-牛顿方程（Gauss-Newton equations）**、**正规方程（normal equations）**）。将方程左边系数矩阵记为$\mathbf{H}$，右边记为$\mathbf{g}$：

$$\mathbf{H} \Delta \mathbf{x} = \mathbf{g} \tag {6-22}$$

对比牛顿法可知，高斯-牛顿法用$\mathbf{J}^{\text{T}} \mathbf{J}$近似二阶Hessian矩阵。

高斯-牛顿算法步骤：

> 1. 给定初始值$\mathbf{x}_{0}$
>
> 2. 第$k$次迭代，计算雅可比矩阵$\mathbf{J}(\mathbf{x}_{k})$和误差$f(\mathbf{x}_{k})$
>
> 3. 求解增量方程：$\mathbf{H} \Delta \mathbf{x}_{k} = \mathbf{g}$
>
> 4. 若$\Delta \mathbf{x}_{k}$足够小，则停止。否则，$\mathbf{x}_{k + 1} = \mathbf{x}_{k} + \Delta \mathbf{x}_{k}$，返回步骤2

原则上，增量方程的求解要求矩阵$\mathbf{H}$是可逆且正定，但实际中$\mathbf{J}^{\text{T}} \mathbf{J}$通常为半正定（奇异矩阵），即*增量方程为病态（ill-condition）*、增量稳定性较差，导致算法不收敛。此外，即使假设$\mathbf{H}$非奇异、非病态，如果求得增量$\Delta \mathbf{x}$太大，也会导致局部近似（一阶泰勒展开（6-19））不够准确，甚至都无法保证目标函数迭代收敛。

非线性优化中，大量高斯-牛顿变种算法尝试改进高斯-牛顿算法的缺点。例如**线搜索方法（line search method）**加入标量$\alpha$，先计算$\Delta \mathbf{x}$，而后进一步计算$\alpha$，使$\| f(\mathbf{x} + \alpha \Delta \mathbf{x}) \|^{2}$最小。

### 6.2.3 列文伯格-马夸尔特法（Levenberg-Marquadt）

一般认为，**列文伯格-马夸尔特法（Levenberg-Marquadt）（阻尼牛顿法（damped Newton method））**比高斯牛顿法更鲁棒，但收敛速度可能慢于高斯牛顿法，在SLAM中大量应用。

高斯牛顿法采用近似二阶泰勒展开只在展开点附近有较好的近似，所以给$\Delta \mathbf{x}$设置**信赖区域（trust region）**，防止其太大导致近似不准确。非线性优化中，称这类方法为**信赖区域方法（trust region method)**。在信赖区域内，近似有效；在信赖区域外，近似可能会产生问题。

信赖区域的范围可根据近似模型与真实函数间的差异确定：如果差异小，就让范围尽可能大；如果差异大，则缩小近似范围。通常采用系数$\rho$判断泰勒近似与真实函数间的差异

$$\rho = \frac{f(\mathbf{x} + \Delta \mathbf{x}) - f(\mathbf{x})}{\mathbf{J}(\mathbf{x}) \Delta \mathbf{x}} \tag {6-23}$$

其中，分子表示真实函数的下降值、分母表示近似模型的下降值。$\rho$越接近1，表示近似越有效；如果$\rho$太小，表示真实下降值远少于近似下降值，近似较差，需要缩小近似范围；反之如果$\rho$较大，表示真实下降值比预期下降值更大，可以增大近似范围。

列文伯格-马夸尔特算法步骤：

> 1. 给定初始值$\mathbf{x}_{0}$、初始半径$\mu$
>
> 2. 第$k$次迭代，计算
>
> $$\min_{\Delta \mathbf{x}_{k}} \frac{1}{2} \| f(\mathbf{x}_{k}) + \mathbf{J}(\mathbf{x}_{k}) \Delta \mathbf{x}_{k} \|^{2}, \text{s.t.} \| \mathbf{D} \Delta \mathbf{x}_{k} \|^{2} \leq \mu \tag {6-24}$$
>
> 其中，$\mu$为信赖区域半径
>
> 3. 计算$\rho$
>
> 4. 若$\rho \gt \frac{3}{4}$，则$\mu = 2 \mu$；若$\rho \lt \frac{1}{2}$，则$\mu = 0.5 \mu$
>
> 5. 若$\rho$大于阈值，表示近似有效，$\mathbf{x}_{k + 1} = \mathbf{x}_{k} + \Delta \mathbf{x}_{k}$
>
> 6. 判断算法是否收敛：收敛则结束；否则返回步骤2

其中，近似范围扩大倍数和阈值为经验值。方程（6－24）中，增量限定在半径为$\mu$的球中，并假定只在球内有效。矩阵$\mathbf{D}$可将增量有效范围修正为椭球。在Levenberg提出的优化方法中，$\mathbf{D}$取为单位阵$\mathbf{I}$，将$\Delta \mathbf{x}$约束在球中。Marqaurdt将$\mathbf{D}$取为非负数对角阵，实际工程中，通常使用$\mathbf{J}^{\text{T}} \mathbf{J}$对角元素平方根，使得在梯度小的维度上约束范围更大。

方程（6－24）为不等式约束优化问题，需要通过`Lagrange`乘子将其转为无约束优化问题：

$$\min_{\Delta \mathbf{x}} \frac{1}{2} \| f(\mathbf{x}_{k}) + \mathbf{J}(\mathbf{x}_{k}) \Delta \mathbf{x}_{k} \|^{2} + \frac{\lambda}{2} \left( \| \mathbf{D} \Delta \mathbf{x}_{k} \|^{2} - \mu \right), \text{s.t.} \lambda \geq 0 \tag {6-25}$$

其中，$\lambda$为`Lagrange`乘子。将计算（6－25）关于$\Delta \mathbf{x}_{k}$的导数并令导数为零，可知该问题的核心仍是计算增量的线性方程：

$$(\mathbf{H} + \lambda \mathbf{D}^{\text{T}} \mathbf{D}) \Delta \mathbf{x} = \mathbf{g} \tag {6-26}$$

相比于高斯-牛顿法，列文伯格-马夸尔特算法的增量方程增加一项$\lambda \mathbf{D}^{\text{T}} \mathbf{D} \Delta \mathbf{x}$。考虑其简化形式，$\mathbf{D} = \mathbf{I}$，则增量方程为：

$$(\mathbf{H} + \lambda \mathbf{I}) \Delta \mathbf{x} = \mathbf{g} \tag {6-26}$$

当$\lambda$较小时，$\mathbf{H}$占主导地位，二次近似模型在该范围内有效，L-M算法与G-N算法相近；反之，$\lambda$较大时，$\lambda \mathbf{I}$占主导地位，L-M算法与一阶梯度下降法（最速下降）相近，表明二次近似无效。L-M算法可在一定程度上避免线性方程组系数矩阵的奇异和病态问题，使增量$\Delta \mathbf{x}$的计算更稳定、准确。

*非线性优化问题的框架分为**Line Search**和**Trust Region**两类。**Line Search**首先确定搜索方向，然后在该方向上寻找步长，以最速下降法和高斯-牛顿法为代表；而**Trust Region**首先确定搜索区域，再查找该区域内的最优点，此类方法以列文伯格-马夸尔特法为代表。*

### 6.2.4 小结

非线性优化问题的通病：目标函数太复杂，导致求解空间上存在大量局部极值点，不同的初始值往往导致计算结果存在差异，大多数算法易陷入局部极小值。视觉SLAM问题中，通常会用ICP、PnP算法提供优化初始值。*良好的初始值对最优化问题非常重要*。

增量方程是一个线性方程，在视觉SLAM中，增量的维度通常成千上万，但其系数矩阵往往有特定的稀疏形式，为实时求解优化问题提供了可能。利用稀疏形式的消元、分解、再求解增量，会让求解的效率极大提高。

## 6.3 实践：Ceres

### 6.3.1 Ceres简介

`Ceres`库用于求解通用最小二乘问题，步骤为：

1. 定义优化问题

2. 设置选项

3. 输入`Ceres`求解

`Ceres`的最小二乘问题标准形式（带边界的核函数最小二乘）：

$$\begin{aligned}
\min_{\mathbf{x}} & \frac{1}{2} \sum_{i} \rho_{i} \left( \| f_{i}(x_{1}, \dots, x_{n}) \|^{2} \right) \\
& \text{s.t.} \lambda \geq 0
\end{aligned} \tag {6-27}$$

可以看到,目标函数由许多平方项,经过一个核函数 ρ(·) 之后,求和组成 ¬ 。在最简
单的情况下,取 ρ 为恒等函数,则目标函数即为许多项的平方和。在这个问题中,优化变
量为 x 1 , . . . , x n ,f i 称为代价函数(Cost function),在 SLAM 中亦可理解为误差项。l j
和 u j 为第 j 个优化变量的上限和下限。在最简单的情况下,取 l j = −∞, u j = ∞(不限
制优化变量的边界),并且取 ρ 为恒等函数时,就得到了无约束的最小二乘问题,和我们
先前说的是一致的。
在 Ceres 中,我们将定义优化变量 x 和每个代价函数 f i ,再调用 Ceres 进行求解。我
¬ 核函数的详细讨论见第十讲。
们可以选择使用 G-N 或者 L-M 进行梯度下降,并设定梯度下降的条件,Ceres 会在优化
之后,将最优估计值返回给我们。下面,我们通过一个曲线拟合的实验,来实际操作一下
Ceres,理解优化的过程。

### 6.3.2 安装Ceres

[Ceres](https://github.com/ceres-solver/ceres-solver)是一个`cmake`工程

1. 依赖项（谷歌的日志、测试工具）：

```sh
# libcxsparse3 for ubuntu 18.04lts (bionic)
sudo apt install liblapack-dev libsuitesparse-dev libcxsparse3 libgflags-dev libgoogle-glog-dev libgtest-dev
```